In [1]:
# !jupyter nbextension enable --py widgetsnbextension --sys-prefix
# !jupyter serverextension enable voila --sys-prefix

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
import cv2

In [4]:
import ipywidgets as widgets
from ipywidgets import Layout, Button, Box, Text

In [5]:
from IPython.display import display, clear_output

In [6]:
from lib.image import image_data
from lib.analysis import dyssynchrony_analysis

In [7]:
import numpy as np
import pandas as pd

In [8]:
import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) 
warnings.filterwarnings("ignore", category=RuntimeWarning) 

In [19]:
def load_cv2_image_from_bytes(bytes_):
    return cv2.imdecode(np.frombuffer(bytes_, dtype=np.uint8), cv2.IMREAD_COLOR)

def get_uploader_content(uploader):
    return uploader.value[list(uploader.value)[0]]["content"]

def display_html(html):
    display(widgets.HTML(value=html))

def display_results(analysis_results):
    """
    Displays the results into the results_box. 
    Every print or display will be automatically redirected to it. 
    """
    results_box.clear_output()
    
    # this is just a sample table...
    display_html("<h2>Slices Table</h2>")
    display(pd.DataFrame(analysis_results["slices"]))

    # ... this is a sample plot
    display_html("<h2>Image Intensities</h2>")
    sns.histplot(pd.DataFrame(analysis_results["image"]["intensities"]))
    
    plt.show()
    
    
    
def clear_uploader(uploader):
    uploader.value.clear()
    uploader._counter = 0

def cardiaAp_analyzer(image):
    return dyssynchrony_analysis.analyze_image(image, \
                                               min_dist_between_maxs.value, \
                                               calibration.value, \
                                               slice_width=slice_width.value)

In [16]:
#Uploader button
uploader = widgets.FileUpload(accept='.tif', multiple=False)

slice_width = widgets.IntText(
    value=1,
    description='Slice width:',
    disabled=False
)

min_dist_between_maxs = widgets.IntText(
    value=0,
    description='Dist between maxs:',
    disabled=False
)

calibration = widgets.IntText(
    value=0,
    description='Calibration:',
    disabled=False
)

In [12]:
html_text_upload = "<p style='font-size:15px; color:black'> Upload you image to initialize the analysis</p>"
header_text_upload = widgets.HTML(value=html_text_upload)

box_upload = Layout(display='flex',
                    flex_flow='flex-wrap',
                    width='50%',
                    margin='0px 10px 10px 0px',
                    padding='0px 25px 25px 25px',
                    justify_content='space-between')

vbox_upload = widgets.VBox([header_text_upload, uploader], layout=box_upload)

In [13]:
box_layout = Layout(display='flex',
                    flex_flow='column',
                    align_items='stretch',
                    width='50%',
                    margin='0px 10px 10px 0px',
                    padding='0px 25px 25px 25px',
                    justify_content='space-between')

vbox_settings = widgets.VBox([slice_width, min_dist_between_maxs, calibration], layout=box_layout)

In [14]:
analyze_btn = widgets.Button(description="Analyze", button_style='danger')
results_box = widgets.Output()

@results_box.capture()
def on_button_clicked(_button):
    global analysis_results
    results_box.clear_output()

    if uploader._counter == 0:
        print("Please upload an image first")
    else:
        print("Anayzing, please wait...")
        analysis_results = cardiaAp_analyzer(load_cv2_image_from_bytes(get_uploader_content(uploader)))
        display_results(analysis_results)
        clear_uploader(uploader)
        
analyze_btn.on_click(on_button_clicked)

In [17]:
clear_uploader(uploader)
display_html("<h1 style='font-size:25px; color:black; margin-left: 15px'> Welcome to Cardi<b>AP</b> </h1>")

display(vbox_upload)
display(vbox_settings)

display(analyze_btn)
display(results_box)

HTML(value="<h1 style='font-size:25px; color:black; margin-left: 15px'> Welcome to Cardi<b>AP</b> </h1>")

Button(button_style='danger', description='Analyze', style=ButtonStyle())

Output(outputs=({'output_type': 'display_data', 'data': {'text/plain': "HTML(value='<h2>Slices Table</h2>')", …